In [1]:
import xgboost as xgb
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
import sys
sys.path.append('../src')

In [3]:
from dataset import load_simple_trig_synth, load_simple_linear_synth, load_exponential_interaction_synth, load_summed_squares_exponential_synth, load_trigonometric_polynomial_synth

c:\Users\evan_\anaconda3\envs\fwal\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
results = {}


datasets = ['simple_trig_synth', 
            'simple_linear_synth', 
            'exponential_interaction_synth',
            'summed_squares_exponential_synth',
            'trigonometric_polynomial_synth',
            ]
for dataset in datasets:
    results[dataset] = []
    if dataset=='simple_trig_synth':
        X, y = load_simple_trig_synth()
    elif dataset=='simple_linear_synth':
        X, y = load_simple_linear_synth()
    elif dataset=='exponential_interaction_synth':
        X, y = load_exponential_interaction_synth()
    elif dataset=='summed_squares_exponential_synth':
        X, y = load_summed_squares_exponential_synth()
    elif dataset=='trigonometric_polynomial_synth':
        X, y = load_trigonometric_polynomial_synth()
        
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    for seed in range(5):
        # Set a seed for reproducibility
        np.random.seed(seed)

        # Sample 1000 random indices
        random_indices = np.random.choice(X_train.index, size=1000, replace=False)

        # Sample the data using the selected indices
        X_train_sampled = X_train.loc[random_indices]
        y_train_sampled = y_train.loc[random_indices]

        # N_ensemble_configurations controls the number of model predictions that are ensembled with feature and class rotations (See our work for details).
        # When N_ensemble_configurations > #features * #classes, no further averaging is applied.
        # Instantiate the XGBoost classifier
        classifier = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

        # Fit the classifier to the sampled training data
        classifier.fit(X_train_sampled, y_train_sampled)

        # Predict the labels of the test set
        y_pred = classifier.predict(X_test)

        acc_score = accuracy_score(y_test, y_pred)
        print('Accuracy', acc_score)
        results[dataset].append(acc_score)

FileNotFoundError: [Errno 2] No such file or directory: '/home/er647/projects/feature-wise-active-learning/data/SyntheticData/simple_trigonometric_synthetic_dataset.csv'

In [26]:
# Convert dictionary to DataFrame
df = pd.DataFrame.from_dict(results, orient='index', columns=[f'seed_{i}' for i in range(5)])
df.reset_index(inplace=True)
df.rename(columns={'index': 'dataset'}, inplace=True)

# Save to CSV
csv_filename = '/home/er647/projects/feature-wise-active-learning/baseline_results/tabpfn_results.csv'  # Saving in the /mnt/data directory
df.to_csv(csv_filename, index=False)

In [27]:
df.head()

,dataset,seed_0,seed_1,seed_2,seed_3,seed_4
0,simple_trig_synth,0.9830,0.9830,0.9830,0.9830,0.9830
1,simple_linear_synth,0.9945,0.9945,0.9945,0.9945,0.9945
2,exponential_interaction_synth,0.7465,0.7465,0.7465,0.7465,0.7465
3,summed_squares_exponential_synth,0.6345,0.6345,0.6345,0.6345,0.6345
4,trigonometric_polynomial_synth,0.9890,0.9890,0.9890,0.9890,0.9890
